# Analysis of Driving Events and generation of Aggregation Functions in Pandas

## Loading Data

In [37]:
#importing necessary packages
import numpy as np
import pandas as pd

In [38]:
df = pd.read_csv('Driving_events.csv')

In [63]:
df.head(30)

,Time,Event
0,0,start_drive
1,1,NaN
2,2,NaN
3,3,NaN
4,4,NaN
5,5,radio_on
6,6,NaN
7,7,NaN
8,8,NaN
9,9,NaN


### Calculation of how many percent of the trip the radio was turned on

In [48]:
df.loc[25, 'Event'] = np.nan

In [67]:
def total_time(df, event_column, start_event, end_event):

    #calculating time of first occurence
    start_times = []
    start_time = df.loc[:,'Time'][df.loc[:, event_column] == start_event]
    start_times.extend(start_time.to_list())


    #calculating time of first opposite occurence
    end_times = []
    end_time = df.loc[:,'Time'][df.loc[:, event_column] == end_event]
    end_times.extend(end_time.to_list())


    time_total = 0

    for n in range(0,len(start_times)):

        #calculating driving length
        time = end_times[n] - start_times[n]
        time_total += time

    return time_total

In [68]:
total_time(df, 'Event', 'window_open', 'window_closed')

10

# Analysis of Driving Events and generation of Aggregation Functions in PySpark

## Loading Data

In [180]:
#importing necessary packages
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import StringType, IntegerType, DoubleType, DecimalType
from decimal import Decimal
from pyspark.sql.window import Window
from pyspark.sql.functions import lit

In [181]:
spark = SparkSession.builder.getOrCreate()

In [182]:
schema = StructType([
    StructField('Time', IntegerType()),
    StructField('Event', StringType()),
])

df = spark.read.csv('Driving_events.csv', header = True, schema = schema)

In [183]:
df.show()

+----+-----------+
|Time|      Event|
+----+-----------+
|   0|start_drive|
|   1|       null|
|   2|       null|
|   3|       null|
|   4|       null|
|   5|   radio_on|
|   6|       null|
|   7|       null|
|   8|       null|
|   9|       null|
|  10|window_open|
|  11|       null|
|  12|       null|
|  13|       null|
|  14|       null|
|  15|       null|
|  16|       null|
|  17|       null|
|  18|       null|
|  19|       null|
+----+-----------+
only showing top 20 rows



In [184]:
df = df.withColumn('VIN', lit('5J8TB4H38Fl002262'))

In [185]:
df.show()

+----+-----------+-----------------+
|Time|      Event|              VIN|
+----+-----------+-----------------+
|   0|start_drive|5J8TB4H38Fl002262|
|   1|       null|5J8TB4H38Fl002262|
|   2|       null|5J8TB4H38Fl002262|
|   3|       null|5J8TB4H38Fl002262|
|   4|       null|5J8TB4H38Fl002262|
|   5|   radio_on|5J8TB4H38Fl002262|
|   6|       null|5J8TB4H38Fl002262|
|   7|       null|5J8TB4H38Fl002262|
|   8|       null|5J8TB4H38Fl002262|
|   9|       null|5J8TB4H38Fl002262|
|  10|window_open|5J8TB4H38Fl002262|
|  11|       null|5J8TB4H38Fl002262|
|  12|       null|5J8TB4H38Fl002262|
|  13|       null|5J8TB4H38Fl002262|
|  14|       null|5J8TB4H38Fl002262|
|  15|       null|5J8TB4H38Fl002262|
|  16|       null|5J8TB4H38Fl002262|
|  17|       null|5J8TB4H38Fl002262|
|  18|       null|5J8TB4H38Fl002262|
|  19|       null|5J8TB4H38Fl002262|
+----+-----------+-----------------+
only showing top 20 rows



### Calculating time between window events

In [186]:
#filtering for window events
df_window = df.where("Event LIKE 'window%'")
#df_window = df.sql("SELECT * FROM df WHERE 'Event' Like 'window%")
df_window.show()

+----+-------------+-----------------+
|Time|        Event|              VIN|
+----+-------------+-----------------+
|  10|  window_open|5J8TB4H38Fl002262|
|  20|window_closed|5J8TB4H38Fl002262|
|  42|  window_open|5J8TB4H38Fl002262|
|  50|window_closed|5J8TB4H38Fl002262|
+----+-------------+-----------------+



In [187]:
#adding column with time_stamp of previous event
df_window_lag = df_window.withColumn('previous_event_time', F.lag(df_window['time']).over(Window.partitionBy("VIN").orderBy("time")))

#calculating time difference between events
df_window_lag = df_window_lag.withColumn('timediff', (df_window_lag['time'] - df_window_lag['previous_event_time']))

#eliminating time the window was closed
df_window_lag = df_window_lag.withColumn('time_open', F.when(df_window_lag['Event'] == 'window_open', 0).otherwise(df_window_lag['timediff']))

df_window_lag.show()

+----+-------------+-----------------+-------------------+--------+---------+
|Time|        Event|              VIN|previous_event_time|timediff|time_open|
+----+-------------+-----------------+-------------------+--------+---------+
|  10|  window_open|5J8TB4H38Fl002262|               null|    null|        0|
|  20|window_closed|5J8TB4H38Fl002262|                 10|      10|       10|
|  42|  window_open|5J8TB4H38Fl002262|                 20|      22|        0|
|  50|window_closed|5J8TB4H38Fl002262|                 42|       8|        8|
+----+-------------+-----------------+-------------------+--------+---------+



In [188]:
df_window_lag.groupBy('VIN').sum().show()

+-----------------+---------+------------------------+-------------+--------------+
|              VIN|sum(Time)|sum(previous_event_time)|sum(timediff)|sum(time_open)|
+-----------------+---------+------------------------+-------------+--------------+
|5J8TB4H38Fl002262|      122|                      72|           40|            18|
+-----------------+---------+------------------------+-------------+--------------+



## Generating Total Time Calculation

In [194]:
def event_info(df, category, activated):

    #filtering for window events
    df_filtered = df.filter(df.Event.startswith(category))

    #adding column with time_stamp of previous event
    df_filtered_lag = df_filtered.withColumn('previous_event_time', F.lag(df_filtered['time']).over(Window.partitionBy("VIN").orderBy("time")))

    #calculating time difference between events
    df_filtered_lag = df_filtered_lag.withColumn('timediff', (df_filtered_lag['time'] - df_filtered_lag['previous_event_time']))

    #eliminating time the window was closed
    df_filtered_lag = df_filtered_lag.withColumn('time_open', F.when(df_filtered_lag['Event'] == activated, 0).otherwise(df_filtered_lag['timediff']))


    #placeholder list
    placeholder_lst = []
    
    #calculating total time active
    total_time_active = df_filtered_lag.select(F.sum('time_open')).collect()[0][0]
    placeholder_lst.append(total_time_active)

    #total time
    total_time = df.select(F.max('Time')).collect()[0][0] - df.select(F.min('Time')).collect()[0][0]
    placeholder_lst.append(total_time)

    #relative time event active
    relative_time = round((total_time_active / total_time), 2)
    placeholder_lst.append(relative_time)

    #assembling final information
    schema1 = StructType([
        StructField('Total Time Active', IntegerType(), True),
        StructField('Total Time', IntegerType(), True),
        StructField('Relative Time Active', DecimalType(), True)
    ])

    #Converting List to RDD
    rdd = spark.sparkContext.parallelize(placeholder_lst)

    #creating data frame
    result = spark.createDataFrame(rdd, schema1)


    return placeholder_lst

In [195]:
df5 = event_info(df, 'window', 'window_open')

In [197]:
print(df5)

[18, 60, 0.3]


In [146]:
total_time_active = df4.select(F.sum('time_open')).collect()[0][0]

In [147]:
print(total_time_active)

18


In [150]:
b = df.select(F.min('Time')).collect()[0][0]

In [151]:
print(b)

0
